In [267]:
import pandas as pd
import numpy as np

In [268]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [269]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [270]:
train = pd.read_csv('/content/drive/MyDrive/project9/train_transformed.csv')
train_labels = pd.read_csv('/content/drive/MyDrive/project9/music30s_trainlabel.csv')
test =pd.read_csv('/content/drive/MyDrive/project9/test_transformed.csv')


In [271]:
train_labels =train_labels.iloc[:,1:]
train_labels.head(3)


,label
0,reggae
1,reggae
2,country




---
### Reading All the Predictions here, in order to stack them and take majority voting along the rows


---



In [272]:
  rfc_pred1 = pd.read_csv('/content/drive/MyDrive/project9/rfc_optuna_pred.csv')

In [273]:
rfc_pred1 =rfc_pred1.rename(columns = {'label':'rfc_label'})

In [274]:
knn_pred1 = pd.read_csv('/content/drive/MyDrive/project9/knn_optuna_pred.csv')

In [275]:
knn_pred1 = knn_pred1.rename(columns = {'label':'knn_label'})

In [276]:
svc_pred1 = pd.read_csv('/content/drive/MyDrive/project9/svc_optuna_pred.csv')

In [277]:

svc_pred1= svc_pred1.rename(columns = {'label':'svc_label'})

In [278]:
logreg_pred1= pd.read_csv('/content/drive/MyDrive/project9/logreg_optuna_pred.csv')

In [279]:
rfc_pred2 = pd.read_csv('/content/drive/MyDrive/project9/FS_FeatSelect/rfc_mutual_pred.csv')

In [280]:
rfc_pred2 =rfc_pred2.rename(columns = {'label':'rfc_fs'})

In [281]:
knn_pred2 = pd.read_csv('/content/drive/MyDrive/project9/FS_FeatSelect/knn_mutual_pred.csv')

In [282]:
knn_pred2 =knn_pred2.rename(columns = {'knn_label':'knn_fs'})

In [283]:
svc_pred2 = pd.read_csv('/content/drive/MyDrive/project9/FS_FeatSelect/svc_mutual_pred.csv')

In [284]:
svc_pred2 =svc_pred2.rename(columns = {'svc_label':'svc_fs'})

In [285]:
logreg_pred2 = pd.read_csv('/content/drive/MyDrive/project9/FS_FeatSelect/logreg_mutual_pred.csv')

In [286]:
logreg_pred2 =logreg_pred2.rename(columns = {'logreg_label':'logreg_fs'})

In [287]:
logreg_pred3 = pd.read_csv('/content/drive/MyDrive/project9/PCA/logreg_pca_pred.csv')

In [288]:
knn_pred3 = pd.read_csv('/content/drive/MyDrive/project9/PCA/knn_pca_pred.csv')

In [289]:
knn_pred3 =knn_pred3.rename(columns = {'knn_label':'knn_pca'})

In [290]:
rfc_pred3 = pd.read_csv('/content/drive/MyDrive/project9/PCA/rfc_pca_pred.csv')

In [291]:
svc_pred3 = pd.read_csv('/content/drive/MyDrive/project9/PCA/svc_pca_pred.csv')



---


Misc ML Algorithms

---



In [292]:
lgbm_pred1 = pd.read_csv('/content/drive/MyDrive/project9/lgbmC_optuna_pred.csv')

In [293]:
xgb_pred1 = pd.read_csv('/content/drive/MyDrive/project9/xgbC_optuna_pred.csv')

In [294]:
lgbm_pred2 = pd.read_csv('/content/drive/MyDrive/project9/FS_FeatSelect/lgbmC_mutual_pred.csv')

In [295]:
xgb_pred2 = pd.read_csv('/content/drive/MyDrive/project9/FS_FeatSelect/xgbC_mutual_pred.csv')

In [296]:
lgbm_pred3 = pd.read_csv('/content/drive/MyDrive/project9/PCA/lgbmC_pca_pred.csv')

In [297]:
xgb_pred3 = pd.read_csv('/content/drive/MyDrive/project9/PCA/xgbC_pca_pred.csv')

In [298]:
display(lgbm_pred3.head())
display(xgb_pred3.head())

,lgbmC_pca
0,hiphop
1,jazz
2,hiphop
3,reggae
4,blues


,xgbC_pca
0,jazz
1,jazz
2,hiphop
3,reggae
4,blues




---
Stacking all the models together


---



---





In [299]:
labels = pd.concat([svc_pred1, rfc_pred1,logreg_pred1,lgbm_pred1,xgb_pred1,
                    svc_pred2, rfc_pred2, knn_pred2, lgbm_pred2, xgb_pred2,
                      svc_pred3, rfc_pred3, xgb_pred3],axis=1)

In [300]:
labels.shape

(100, 13)

In [301]:
labels.head()

,svc_label,rfc_label,logreg_label,lgbmC_label,xgbC_label,svc_fs,rfc_fs,knn_fs,lgbmC_fs,xgbC_fs,svc_pca,rfc_pca,xgbC_pca
0,country,reggae,reggae,reggae,reggae,blues,reggae,country,pop,reggae,country,reggae,jazz
1,jazz,classical,classical,classical,jazz,classical,classical,classical,classical,classical,jazz,jazz,jazz
2,hiphop,hiphop,hiphop,hiphop,hiphop,hiphop,hiphop,hiphop,hiphop,hiphop,hiphop,hiphop,hiphop
3,reggae,country,reggae,rock,rock,pop,country,reggae,rock,country,reggae,reggae,reggae
4,blues,blues,blues,blues,blues,reggae,rock,rock,blues,blues,blues,blues,blues


In [302]:
from collections import Counter
def mode(data):
    """Return the most common data point from discrete or nominal data.
    ``mode`` assumes discrete data, and returns a single value. This is the
    standard treatment of the mode as commonly taught in schools:
        >>> mode([1, 1, 2, 3, 3, 3, 3, 4])
        3
    This also works with nominal (non-numeric) data:
        >>> mode(["red", "blue", "blue", "red", "green", "red", "red"])
        'red'
    If there are multiple modes with same frequency, return the first one
    encountered:
        >>> mode(['red', 'red', 'green', 'blue', 'blue'])
        'red'
    If *data* is empty, ``mode``, raises StatisticsError.
    """
    pairs = Counter(iter(data)).most_common(1)
    try:
        return pairs[0][0]
    except IndexError:
        raise StatisticsError('no mode for empty data') from None



In [303]:

preds= pd.DataFrame()
preds['label'] = labels.mode(axis=1).iloc[:, 0]

In [304]:

preds.head()

,label
0,reggae
1,classical
2,hiphop
3,reggae
4,blues


In [305]:
preds.to_csv('music_genre_predictions.csv', index=False)

In [306]:
!cp -r '/content/music_genre_predictions.csv' /content/drive/MyDrive/project9/Misc
